In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer, T5ForConditionalGeneration, T5Tokenizer
import torch

In [ ]:
qa_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
qa_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
! pip install sentencepiece


In [ ]:
summarization_model = T5ForConditionalGeneration.from_pretrained('t5-small')
summarization_tokenizer = T5Tokenizer.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
wikipedia_text="""BERT (Bidirectional Encoder Representations from Transformers) is a foundational model in the realm of natural language processing (NLP), specifically known for its ability to understand and process language bidirectionally, meaning it takes into account both the left and right context of a word in a sentence. Unlike traditional models that read text sequentially, BERT uses transformers, which allow it to focus on different parts of a sentence simultaneously, greatly improving performance on tasks like question answering, sentiment analysis, and text classification. BERT's pre-training on large corpora and its fine-tuning on specific downstream tasks marked a significant leap in NLP. This development paved the way for the rise of large language models (LLMs), which, by leveraging massive datasets and extensive training, can generate highly coherent and context-aware text. LLMs, including models like GPT (Generative Pre-trained Transformer), take the transformer architecture to even larger scales, enabling capabilities beyond sentence-level tasks to generate long-form text, engage in dialogue, and perform complex reasoning. The combination of BERT's bidirectional attention and the generative capabilities of LLMs like GPT has revolutionized AI's ability to understand, interpret, and generate human language, powering applications ranging from chatbots to content generation, summarization, and more. """

In [ ]:
def answer_question(question, text):
    inputs = qa_tokenizer.encode_plus(question, text, return_tensors='pt', max_length=512, truncation=True)
    input_ids = inputs['input_ids'].tolist()[0]

    outputs = qa_model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1

    answer = qa_tokenizer.convert_tokens_to_string(qa_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return answer

In [ ]:

def summarize_answer(answer_text):
    # Encode the input (answer text)
    inputs = summarization_tokenizer.encode("summarize: " + answer_text, return_tensors="pt", max_length=512, truncation=True)
    # Generate summary
    summary_ids = summarization_model.generate(inputs, max_length=50, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
question = "how it works?'"
extracted_answer = answer_question(question, wikipedia_text)

In [ ]:
summarized_answer = summarize_answer(extracted_answer)

In [ ]:

print(summarized_answer)

bert uses transformers to focus on different parts of a sentence simultaneously. takes into account both the left and right context of a word in a sentence.
